In [1]:
import pandas as pd
import numpy as np

In [2]:
# Step 1: Read in the first 200K lines, with 8 columns
df = pd.read_csv('C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\ebird-project\\data\\ebd_relJan-2020.txt', sep='\t', nrows=200000, usecols=['COMMON NAME', 'COUNTRY', 'STATE', 'COUNTY', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE', 'OBSERVATION COUNT'])

print(df.shape)

df.head()

(200000, 8)


,COMMON NAME,OBSERVATION COUNT,COUNTRY,STATE,COUNTY,LATITUDE,LONGITUDE,OBSERVATION DATE
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27
1,White-rumped Sandpiper,4,Canada,Quebec,Manicouagan,49.216670,-68.150000,1993-11-07
2,Common Scoter,1,Sweden,Hallands län [SE-13],NaN,57.065084,12.243579,1998-02-21
3,Ring-billed Gull,15,Canada,Manitoba,South Interlake,50.193256,-97.137935,1985-04-14
4,Red-winged Blackbird,500,Canada,Manitoba,South Interlake,50.193256,-97.137935,1986-09-01


In [3]:
df.isnull().sum()

COMMON NAME              0
OBSERVATION COUNT        0
COUNTRY                  0
STATE                    0
COUNTY               29261
LATITUDE                 0
LONGITUDE                0
OBSERVATION DATE         0
dtype: int64

In [4]:
# Rename columns for ease of use
df.rename(columns={
    'COMMON NAME': 'name',
    'OBSERVATION COUNT': 'observ_count',
    'COUNTRY': 'country',
    'STATE': 'state',
    'COUNTY': 'county',
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude',
    'OBSERVATION DATE': 'observ_date'
}, inplace=True)

In [5]:
# Filter for just US birds
us_birds = df.query("country == 'United States'")

print(us_birds.shape)
us_birds.head()

(105294, 8)


,name,observ_count,country,state,county,latitude,longitude,observ_date
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27
6,Greater Yellowlegs,X,United States,Texas,Aransas,28.240392,-96.818819,1986-04-06
12,White-crowned Sparrow,X,United States,Arizona,Cochise,31.898164,-109.115932,1998-11-27
13,Green-winged Teal,11,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18
14,Yellow-rumped Warbler,5,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18


In [6]:
us_birds = us_birds.copy()

# Drop missing column values
us_birds.dropna(subset=['county'], inplace=True)

us_birds.isnull().sum()

name            0
observ_count    0
country         0
state           0
county          0
latitude        0
longitude       0
observ_date     0
dtype: int64

In [9]:
us_birds['bad_name'] = us_birds['name'].apply(lambda x: 0 if ("sp." in x) or ("(" in x) or ("/" in x) else 1)
us_birds.bad_name.value_counts()

1    104333
0       744
Name: bad_name, dtype: int64

In [10]:
mask = us_birds['bad_name'] == 0
us_birds = us_birds[~mask]
us_birds.shape

(104333, 9)

In [11]:
# Replace 'X' in 'observ_count' with 1
us_birds['observ_count'] = us_birds['observ_count'].apply(lambda x: 1 if x == 'X' else x)

us_birds.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,bad_name
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1
6,Greater Yellowlegs,1,United States,Texas,Aransas,28.240392,-96.818819,1986-04-06,1
12,White-crowned Sparrow,1,United States,Arizona,Cochise,31.898164,-109.115932,1998-11-27,1
13,Green-winged Teal,11,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1
14,Yellow-rumped Warbler,5,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1


In [12]:
# Convert 'observ_date' to datetime and extract year and month
us_birds.observ_date = pd.to_datetime(us_birds['observ_date'], infer_datetime_format=True)

us_birds['month'] = us_birds.observ_date.dt.month

us_birds.shape

(104333, 10)

In [13]:
# 'season' column from month values
def month_to_season(x):
    if x in [12, 1, 2]:
        return 'Winter'
    elif x in [3, 4, 5]:
        return 'Spring'
    elif x in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

us_birds['season'] = us_birds['month'].apply(month_to_season)
us_birds.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,bad_name,month,season
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1,8,Summer
6,Greater Yellowlegs,1,United States,Texas,Aransas,28.240392,-96.818819,1986-04-06,1,4,Spring
12,White-crowned Sparrow,1,United States,Arizona,Cochise,31.898164,-109.115932,1998-11-27,1,11,Fall
13,Green-winged Teal,11,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1,12,Winter
14,Yellow-rumped Warbler,5,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1,12,Winter


In [14]:
us_birds.season.value_counts()['Spring']

39395

In [15]:
# Create merge column
us_birds['county_state'] = us_birds['county'] + us_birds['state']

us_birds.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,bad_name,month,season,county_state
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1,8,Summer,CookIllinois
6,Greater Yellowlegs,1,United States,Texas,Aransas,28.240392,-96.818819,1986-04-06,1,4,Spring,AransasTexas
12,White-crowned Sparrow,1,United States,Arizona,Cochise,31.898164,-109.115932,1998-11-27,1,11,Fall,CochiseArizona
13,Green-winged Teal,11,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1,12,Winter,AdaIdaho
14,Yellow-rumped Warbler,5,United States,Idaho,Ada,43.609793,-116.206427,1982-12-18,1,12,Winter,AdaIdaho


In [16]:
us_birds.shape

(104333, 12)

In [17]:
# Read in regions df
regions = pd.read_excel("C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\ebird-project\data\\URAmericaMapCountyList.xlsx", skiprows=3, usecols=['State', 'CountyName', 'RegionName'])

print(regions.shape)

regions.head()

(3142, 3)


,State,CountyName,RegionName
0,Alabama,"Autauga County, Alabama",01 Deep South
1,Alabama,"Baldwin County, Alabama",03 Gulf Coast
2,Alabama,"Barbour County, Alabama",01 Deep South
3,Alabama,"Bibb County, Alabama",02 Appohzarka
4,Alabama,"Blount County, Alabama",02 Appohzarka


In [18]:
# Strip leading whitespace
regions.State = regions.State.str.strip()

regions.State.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [19]:
# Split region numbers off RegionName
regions['RegionName'] = regions['RegionName'].apply(lambda x: ' '.join(x.split()[1:]))

In [20]:
# Split state name off county
regions['CountyName'] = regions['CountyName'].apply(lambda x: x.split(',')[0])

regions.head()

,State,CountyName,RegionName
0,Alabama,Autauga County,Deep South
1,Alabama,Baldwin County,Gulf Coast
2,Alabama,Barbour County,Deep South
3,Alabama,Bibb County,Appohzarka
4,Alabama,Blount County,Appohzarka


In [21]:
# Change Alaska county names to match birds counties
county_dict = {
    'Aleutians East Borough': 'Aleutians East',
    'Aleutians West Census Area': 'Aleutians West',
    'Anchorage Municipality': 'Anchorage',
    'Bethel Census Area': 'Bethel',
    'Bristol Bay Borough': 'Bristol Bay',
    'Denali Borough': 'Denali',
    'Dillingham Census Area': 'Dillingham',
    'Fairbanks North Star Borough': 'Fairbanks North Star',
    'Haines Borough': 'Haines',
    'Hoonah-Angoon Census Area': 'Skagway-Hoonah-Angoon',
    'Juneau City and Borough': 'Juneau',
    'Kenai Peninsula Borough': 'Kenai Peninsula',
    'Ketchikan Gateway Borough': 'Ketchikan Gateway',
    'Kodiak Island Borough': 'Kodiak Island',
    'Kusilvak Census Area': 'Kusilvak',
    'Lake and Peninsula Borough': 'Lake and Peninsula',
    'Matanuska-Susitna Borough': 'Matanuska-Susitna',
    'Nome Census Area': 'Nome',
    'North Slope Borough': 'North Slope',
    'Northwest Arctic Borough': 'Northwest Arctic',
    'Petersburg Borough': 'Petersburg Borough',
    'Prince of Wales-Hyder Census Area': 'Prince of Wales-Outer Ketchikan',
    'Sitka City and Borough': 'Sitka',
    'Southeast Fairbanks Census Area': 'Southeast Fairbanks',
    'Valdez-Cordova Census Area': 'Valdez-Cordova',
    'Yakutat City and Borough': 'Yakutat',
    'Yukon-Koyukuk Census Area': 'Yukon-Koyukuk'
}

regions['CountyName'] = regions['CountyName'].apply(lambda x: county_dict[x] if x in county_dict.keys() else x)
regions.CountyName.nunique()

1877

In [22]:
# Remove 'Parish' from CountyName
regions['CountyName'] = regions['CountyName'].apply(lambda x: x if 'Parish' not in x else ' '.join(x.split()[:-1]))
regions.query("State == 'Louisiana'")

,State,CountyName,RegionName
1113,Louisiana,Acadia,Gulf Coast
1114,Louisiana,Allen,Deep South
1115,Louisiana,Ascension,Gulf Coast
1116,Louisiana,Assumption,Gulf Coast
1117,Louisiana,Avoyelles,Gulf Coast
...,...,...,...
1172,Louisiana,Webster,Deep South
1173,Louisiana,West Baton Rouge,Gulf Coast
1174,Louisiana,West Carroll,Deep South
1175,Louisiana,West Feliciana,Gulf Coast


In [23]:
# Capitalize 'city' in 'Richmond city'
print(regions.at[2944, 'CountyName'])
regions.at[2944, 'CountyName'] = 'Richmond City'
regions.at[2944, 'CountyName']

Richmond city


'Richmond City'

In [24]:
# Capitalize 'city' in 'St. Louis city'
print(regions.at[1597, 'CountyName'])
regions.at[1597, 'CountyName'] = 'St. Louis City'
regions.at[1597, 'CountyName']

St. Louis city


'St. Louis City'

In [25]:
# Split off 'county' from name
regions['CountyName'] = regions['CountyName'].apply(lambda x: x if 'County' not in x else ' '.join(x.split()[:-1]))

regions.head()

,State,CountyName,RegionName
0,Alabama,Autauga,Deep South
1,Alabama,Baldwin,Gulf Coast
2,Alabama,Barbour,Deep South
3,Alabama,Bibb,Appohzarka
4,Alabama,Blount,Appohzarka


In [26]:
# Combine 'county' and 'state' for merge
regions['county_state'] = regions['CountyName'] + regions.State

In [27]:
print("Regions shape:", regions.shape)
print("Birds shape:", us_birds.shape)

Regions shape: (3142, 4)
Birds shape: (104333, 12)


In [28]:
merged = us_birds.merge(regions)
merged.shape

(103992, 15)

### Below here figuring out proper ratios for the labels

In [30]:
merged.name.value_counts(normalize=True)

Mourning Dove        0.016078
American Robin       0.014886
American Crow        0.014626
European Starling    0.013665
Song Sparrow         0.013616
                       ...   
Pechora Pipit        0.000010
Azure Gallinule      0.000010
Hawaiian Petrel      0.000010
Baird's Sparrow      0.000010
Green Parakeet       0.000010
Name: name, Length: 790, dtype: float64

In [34]:
counts = merged.name.value_counts(normalize=True)

print("Birds that appear more than .01% of the time", len(counts[counts > .001]))

print("Birds that appear more than .05% of the time", len(counts[counts > .005]))

print("Birds that appear more than .001% of the time", len(counts[counts > .0001]))

Birds that appear more than .01% of the time 252
Birds that appear more than .05% of the time 47
Birds that appear more than .001% of the time 537


In [35]:
# Bird rarity as a function of region and season

#table = pd.pivot_table(df, values='D', index=['A', 'B'],
                   # columns=['C'], aggfunc=np.sum)
season_region_ct = pd.pivot_table(merged, index='name', columns=['RegionName', 'season'], values='observ_count', aggfunc='count', fill_value=0.0)
season_region_ct

RegionName              Appohzarka                      Deep South         \
season                        Fall Spring Summer Winter       Fall Spring   
name                                                                        
Abert's Towhee                   0      0      0      0          0      0   
Acadian Flycatcher               1     10     12      0          1     13   
Acorn Woodpecker                 0      0      0      0          0      0   
African Silverbill               0      0      0      0          0      0   
Akiapolaau                       0      0      0      0          0      0   
...                            ...    ...    ...    ...        ...    ...   
Yellow-rumped Warbler           18     26      0      8         18     31   
Yellow-throated Vireo            3     10      7      0          6     24   
Yellow-throated Warbler          2     14      1      0          1     13   
Zebra Dove                       0      0      0      0          0      0   
Zone-tailed Hawk                 0      0      0      0          0      0   

RegionName                            Great Lakes         ...  \
season                  Summer Winter        Fall Spring  ...   
name                                                      ...   
Abert's Towhee               0      0           0      0  ...   
Acadian Flycatcher           3      0           1      6  ...   
Acorn Woodpecker             0      0           0      0  ...   
African Silverbill           0      0           0      0  ...   
Akiapolaau                   0      0           0      0  ...   
...                        ...    ...         ...    ...  ...   
Yellow-rumped Warbler        0     33          12     39  ...   
Yellow-throated Vireo        8      0           3      8  ...   
Yellow-throated Warbler      4      1           0      2  ...   
Zebra Dove                   0      0           0      0  ...   
Zone-tailed Hawk             0      0           0      0  ...   

RegionName              Southern Florida        Southwest                \
season                            Summer Winter      Fall Spring Summer   
name                                                                      
Abert's Towhee                         0      0         6      5     10   
Acadian Flycatcher                     1      0         0      3      0   
Acorn Woodpecker                       0      0         6     12     15   
African Silverbill                     0      0         0      0      0   
Akiapolaau                             0      0         0      0      0   
...                                  ...    ...       ...    ...    ...   
Yellow-rumped Warbler                  0      8        18     28      2   
Yellow-throated Vireo                  0      0         0      0      0   
Yellow-throated Warbler                1      4         0      1      0   
Zebra Dove                             0      0         0      0      0   
Zone-tailed Hawk                       0      0         0      7      4   

RegionName                     West Coast                       
season                  Winter       Fall Spring Summer Winter  
name                                                            
Abert's Towhee              13          1      1      0      1  
Acadian Flycatcher           0          0      0      0      0  
Acorn Woodpecker             5          7     10     10      5  
African Silverbill           0          1      0      0      0  
Akiapolaau                   0          0      0      1      0  
...                        ...        ...    ...    ...    ...  
Yellow-rumped Warbler       20         39     23      4     28  
Yellow-throated Vireo        0          0      0      0      0  
Yellow-throated Warbler      0          0      0      0      0  
Zebra Dove                   0          1      1      5      2  
Zone-tailed Hawk             0          0      0      0      0  

[790 rows x 56 columns]

In [47]:
# Bird rarity by region and season
def season_region_bird_rarity(bird, region, season):
    bird_percent = season_region_ct[(region, season)][bird] / season_region_ct[(region, season)].sum()
    if bird_percent > 0.005:
        return "Common"
    elif bird_percent > 0.001:
        return "Uncommon"
    else:
        return "Rare"
    
merged['bird_rareness'] = merged.apply(lambda x: season_region_bird_rarity(x['name'], x['RegionName'], x['season']), axis=1)
merged.bird_rareness.value_counts(normalize=True)

Common      0.654406
Uncommon    0.304495
Rare        0.041099
Name: bird_rareness, dtype: float64

In [48]:
merged.bird_rareness.value_counts(normalize=True)['Uncommon']

0.3044945765058851

In [44]:
merged[merged['bird_rareness'] == 'Rare']

,name,observ_count,country,state,county,latitude,longitude,observ_date,bad_name,month,season,county_state,State,CountyName,RegionName,seas_reg_rare,bird_rareness
10,Gray-cheeked Thrush,15,United States,Illinois,Cook,41.850030,-87.650050,1957-05-18,1,5,Spring,CookIllinois,Illinois,Cook,Great Lakes,Uncommon,Rare
28,Eurasian Wigeon,1,United States,Illinois,Cook,41.707907,-87.886848,1939-04-30,1,4,Spring,CookIllinois,Illinois,Cook,Great Lakes,Rare,Rare
52,Monk Parakeet,2,United States,Illinois,Cook,41.775629,-87.583273,1994-04-04,1,4,Spring,CookIllinois,Illinois,Cook,Great Lakes,Rare,Rare
80,Wilson's Phalarope,2,United States,Illinois,Cook,41.850030,-87.650050,1972-05-27,1,5,Spring,CookIllinois,Illinois,Cook,Great Lakes,Rare,Rare
88,Vermilion Flycatcher,1,United States,Illinois,Cook,41.850030,-87.650050,1986-04-16,1,4,Spring,CookIllinois,Illinois,Cook,Great Lakes,Rare,Rare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103934,American Black Duck,1,United States,North Dakota,Towner,48.861309,-99.177897,1999-06-18,1,6,Summer,TownerNorth Dakota,North Dakota,Towner,Heartland,Rare,Rare
103938,Gray-cheeked Thrush,1,United States,North Carolina,Iredell,35.787487,-80.828693,1997-10-04,1,10,Fall,IredellNorth Carolina,North Carolina,Iredell,Deep South,Uncommon,Rare
103948,Broad-winged Hawk,1,United States,Iowa,Boone,41.980503,-93.892694,1992-07-21,1,7,Summer,BooneIowa,Iowa,Boone,Heartland,Uncommon,Rare
103984,Blue Grosbeak,1,United States,Missouri,Cole,38.602875,-92.304467,1992-06-17,1,6,Summer,ColeMissouri,Missouri,Cole,Heartland,Uncommon,Rare
